In [1]:
import PIL.Image as Image
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import h5py
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
import numpy as np
import PIL.Image as Image

In [2]:
# Check if data is correct
img = np.array(Image.open("./content/datasets/data_in/objects/real/segm/0000/000000.png"))
obj_ids = np.unique(img)
print(obj_ids)
obj_id = obj_ids[0]
class_id = obj_id // 1000
print(class_id)
obj_instance_id = obj_id % 1000
print(obj_instance_id)

[  0   1   2   3   5   6   7   8   9  10  11  12  13  18 255]
0
0


In [6]:
unique=0

def map_to_r(number):
    colorr = [255, 128, 0, 0, 128, 0, 0, 0, 128, 0, 0, 0, 128, 0, 128, 255, 128, 255, 0, 128]
    if number<20:
      return int(colorr[number])
    else:
      return 0

def map_to_g(number):

    colorg = [0, 128, 255, 128, 0, 0, 128, 0, 0, 128, 0, 128, 0, 128, 255, 128, 0, 128, 255, 128]

    if number<20:
      return int(colorg[number])
    else:
      return 0

def map_to_b(number):

    colorb = [0, 0, 0, 128, 128, 255, 255, 128, 0, 128, 128, 255, 128, 0, 0, 0, 128, 255, 255, 128]

    if number<20:
      return int(colorb[number])
    else:
      return 255

def parse_txt_file(txt_file):
    color_to_class_id = {}
    with open(txt_file, 'r') as f:
        lines = f.readlines()
        for line in lines[1:]:
            parts = line.strip().split()
            class_name = parts[0]
            rgb_values = tuple(map(int, parts[1:]))
            color_to_class_id[rgb_values] = class_name
    return color_to_class_id

def rgb_mask_to_class_ids(rgb_mask, class_to_id, label_id_mapping):
    lookup_table = np.zeros((256, 256, 256), dtype=np.uint8)
    for rgb_pixel, class_name in class_to_id.items():
        if class_name.startswith("Car") or class_name.startswith("Van"):
            class_name = "Car"
        lookup_table[rgb_pixel] = label_id_mapping[class_name]
    flat_rgb_mask = rgb_mask.reshape(-1, 3)
    class_ids = lookup_table[flat_rgb_mask[:, 0], flat_rgb_mask[:, 1], flat_rgb_mask[:, 2]]
    class_ids = class_ids.reshape(rgb_mask.shape[:2])

    return class_ids

def interpolate_color(index, unique):
    r = int(index * (255 / unique))
    g = int((unique - index) * (255 / unique))
    b = 128
    return r, g, b


def load_sim_data(dataset_index,weather="clone",visualize=False,stop=0,color_map=False):
  images_list_sim = []
  bounding_boxes_list_sim = []
  segmentation_masks_list_sim = []
  images_list_index_sim = []
  semantic_id_list_sim = []
  raw_image_sim = []

  label_id_mapping = {
        'Road': 0,
        'Sidewalk': 1,
        'Building': 2,
        'Wall': 2,
        'Fence': 2,
        'Pole': 2,
        'TrafficLight': 6,
        'TrafficSign': 7,
        'Vegetation': 8,
        'Tree': 8,
        'Terrain': 1,
        'Sky': 10,
        'Person': 11,
        'Rider': 12,
        'Car': 13,
        'Truck': 14,
        'Bus': 15,
        'Train': 16,
        'Motorcycle': 17,
        'Bicycle': 18,
        'Void': 255,
        'Misc': 2,
        'GuardRail': 21,
    }
  images_folder = f"./content/datasets/data_in/objects/sim/img/{dataset_index}/clone"
  labels_file = f"./content/datasets/data_in/objects/sim/obj/{dataset_index}_clone.txt"
  segmentation_masks_folder = f"./content/datasets/data_in/objects/sim/segm/{dataset_index}/clone"
  txt_file_path = f'./content/datasets/data_in/encodings/{dataset_index}_clone_scenegt_rgb_encoding.txt'
  color_to_class_id_mapping = parse_txt_file(txt_file_path)

  # Read the ground truth labels from the text file
  with open(labels_file, 'r') as f:
      lines = f.readlines()[1:]

  ground_truth_data = {}
  image_files = sorted(os.listdir(images_folder))
  max_index=int(image_files[-1].strip().split('.png')[0])
  for index in range(0,max_index+1):
    if index not in ground_truth_data:
                  ground_truth_data[index] = []

  # Process ground truth lines and group them by image index
  for line in lines:
      line_parts = line.strip().split(' ')
      image_index = int(line_parts[0])

      if(image_index<=stop or stop==0):
          label = line_parts[2]

          if label != 'DontCare':
              x_min = int(float(line_parts[6]))
              y_min = int(float(line_parts[7]))
              x_max = int(float(line_parts[8]))
              y_max = int(float(line_parts[9]))
              ground_truth_data[image_index].append((label, x_min, y_min, x_max, y_max))

  # Process images in the sorted order
  for image_index in sorted(ground_truth_data.keys()):
      if(image_index%50==0):
        print(image_index)
      image_file = sorted(os.listdir(images_folder))[image_index]
      image_path = os.path.join(images_folder, image_file)
      image = cv2.imread(image_path)
      raw_image_sim.append(image)
      bounding_boxes = ground_truth_data[image_index]
      image2 = image.copy()

      for label, x_min, y_min, x_max, y_max in bounding_boxes:
          if x_max > x_min and y_max > y_min:
              cv2.rectangle(image2, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
              cv2.putText(image2, label, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
              bounding_boxes_list_sim.append((image_index,label, x_min, y_min, x_max, y_max))

      segmentation_mask_file = f"{os.path.splitext(image_file)[0]}.png"
      segmentation_mask_path = os.path.join(segmentation_masks_folder, segmentation_mask_file)
      if os.path.exists(segmentation_mask_path):
          segmentation_mask = cv2.imread(segmentation_mask_path)
          images_list_index_sim.append(image_index)
          images_list_sim.append(image2)
          semantic_id = rgb_mask_to_class_ids(segmentation_mask[:,:,::-1], color_to_class_id_mapping,label_id_mapping)
          if(color_map):
            colored_mask = np.zeros_like(image)
            colored_mask[:, :, 0] = np.vectorize(map_to_b)(semantic_id)
            colored_mask[:, :, 1] = np.vectorize(map_to_g)(semantic_id)
            colored_mask[:, :, 2] = np.vectorize(map_to_r)(semantic_id)
            segmentation_masks_list_sim.append(colored_mask)
            if(visualize):
              overlay = cv2.addWeighted(image, 0.7, segmentation_mask, 0.3, 0)
              plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
              plt.show()
              plt.imshow(cv2.cvtColor(segmentation_mask, cv2.COLOR_BGR2RGB))
              plt.show()
              plt.imshow(cv2.cvtColor(colored_mask, cv2.COLOR_BGR2RGB))
              plt.show()

          semantic_id_list_sim.append(semantic_id)
  return images_list_index_sim,images_list_sim,segmentation_masks_list_sim,semantic_id_list_sim,bounding_boxes_list_sim,raw_image_sim

In [ ]:
images_list_sim = {}
bounding_boxes_list_sim = {}
segmentation_masks_list_sim = {}
images_list_index_sim = {}
semantic_id_list_sim = {}
raw_image_sim = {}

dataset_index_list=["0001","0002","0006","0018","0020"]

for dataset_index in dataset_index_list:
  print("-STARTING sim- ", dataset_index)

  images_list_index_sim_out,images_list_sim_out,segmentation_masks_list_sim_out,semantic_id_list_sim_out,bounding_boxes_list_sim_out,raw_image_sim_out=load_sim_data(dataset_index,visualize=False,stop=0,color_map=True)
  images_list_sim[dataset_index] = images_list_sim_out
  bounding_boxes_list_sim[dataset_index] = bounding_boxes_list_sim_out
  segmentation_masks_list_sim[dataset_index] = segmentation_masks_list_sim_out
  images_list_index_sim[dataset_index] = images_list_index_sim_out
  semantic_id_list_sim[dataset_index] = semantic_id_list_sim_out
  raw_image_sim[dataset_index] = raw_image_sim_out
  print("-DONE- ", dataset_index)

In [7]:
import pickle
!mkdir outputs
with open('./content/datasets/pkl_out/images_list_sim.pkl', 'wb') as file:
    pickle.dump(images_list_sim, file)

with open('./content/datasets/pkl_out/bounding_boxes_list_sim.pkl', 'wb') as file:
    pickle.dump(bounding_boxes_list_sim, file)

with open('./content/datasets/pkl_out/segmentation_masks_list_sim.pkl', 'wb') as file:
    pickle.dump(segmentation_masks_list_sim, file)

with open('./content/datasets/pkl_out/semantic_id_list_sim.pkl', 'wb') as file:
    pickle.dump(semantic_id_list_sim, file)
with open('./content/datasets/pkl_out/raw_image_sim.pkl', 'wb') as file:
    pickle.dump(raw_image_sim, file)

In [6]:
# with open('./content/datasets/pkl_out/raw_image_sim.pkl', 'rb') as file:
#     loaded_dictionary_images_sim = pickle.load(file)
# with open('./content/datasets/pkl_out/segmentation_masks_list_sim.pkl', 'rb') as file:
#     loaded_segmentation_masks_sim = pickle.load(file)
# with open('./content/datasets/pkl_out/semantic_id_list_sim.pkl', 'rb') as file:
#     loaded_semantic_id_sim = pickle.load(file)

loaded_segmentation_masks_sim=segmentation_masks_list_sim
loaded_semantic_id_sim=semantic_id_list_sim
loaded_dictionary_images_sim=raw_image_sim

In [ ]:
def save_dictionary_to_h5(file_path, data_dict):
    with h5py.File(file_path, 'w') as hf:
        for key, array_list in data_dict.items():
            # Create a group for each key in the dictionary
            group = hf.create_group(key)
            for i, arr in enumerate(array_list):
                # Save each array as a dataset within the group
                group.create_dataset(f'array_{i}', data=arr)


def save_dictionary_to_h5_bounding(file_path, data_dict):
    with h5py.File(file_path, 'w') as hf:
        for key, array_list in data_dict.items():
            # Create a group for each key in the dictionary
            group = hf.create_group(key)
            for i, arr in enumerate(array_list):
                output=[0,0,0,0,0,0]
                if(arr[1]=='Car'):
                  output[1]=3
                else:
                  output[1]=0
                output[2]=arr[2]
                output[3]=arr[3]
                output[4]=arr[4]
                output[5]=arr[5]
                output[0]=arr[0]
                group.create_dataset(f'array_{i}', data=output)

file_path = './content/datasets/h5_out2/raw_image_sim.h5'
save_dictionary_to_h5(file_path, loaded_dictionary_images_sim)

file_path = './content/datasets/h5_out2/segmentation_masks_list_sim.h5'
save_dictionary_to_h5(file_path, loaded_segmentation_masks_sim)

file_path = './content/datasets/h5_out2/semantic_id_list_sim.h5'
save_dictionary_to_h5(file_path, loaded_semantic_id_sim)
print(bounding_boxes_list_sim)

file_path = './content/datasets/h5_out2/bounding_boxes_sim.h5'
save_dictionary_to_h5_bounding(file_path, bounding_boxes_list_sim)
